# Tokenizer 사용하기

주어진 텍스트를 머신에게 이해시키려면 먼저 어떤 단위로 다룰 것인지를 정해야 합니다.

이러한 단위로 쪼개진 것을 토큰(Token)이라고 하며, 텍스트를 토큰화하는 것을 토크나이저 (Tokenizer)라고 합니다.

토큰은 띄어쓰기, 단어 단위로 이루어질 수도 있으며, (한글의 경우) 형태소 단위로 이루어집니다.

해당 코드는 영어에서는 단어 단위 토크나이저, 한글에서는 형태소 단위 토크나이저를 소개합니다.


### 패키지 다운로드

해당 강의의 모든 코드는 Pytorch 패키지를 [홈페이지](https://pytorch.org/get-started/locally/)에서 이미 적절한 방식으로 다운로드 받았다고 가정합니다.

- pandas
- spacy (다국어 토크나이저)
- python-mecab-ko (한국어 형태소 토크나이저)


In [ ]:
%pip install pandas spacy python-mecab-ko
# Spacy Pipeline download
!python -m spacy download en_core_web_sm

## 영어 토큰화

spacy 클래스를 활용하여 다음과 같이 영어 텍스트를 토큰화합니다.

In [ ]:
import spacy
spacy_en = spacy.load("en_core_web_sm")

en_text = "The quick brown fox jumps over the lazy dog."

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

print(tokenize_en(en_text))

## 한글 토큰화

한글 형태소 토큰화에는 KoNLPy와 MeCab-ko를 주로 사용하며, 여기서는 Mecab-Ko 구현 패키지 python-mecab-ko를 사용해봅시다.

MeCab-ko 패키지는 일본의 형태소 분석을 위해 만들어진 텍스트 분할 라이브러리 MeCab을 한국어에 맞게 변경한 것입니다.

In [ ]:
from mecab import MeCab

ko_text = "추운 겨울에는 따뜻한 커피와 티를 마셔야지요."

mecab = MeCab()

print(mecab.morphs(ko_text))

# 단어 집합 (vocabulary)생성

"네이버 영화 리뷰 분류하기" 데이터를 이용하여 사전 만들기

In [ ]:
import urllib.request
import pandas as pd
import re
from collections import Counter

# 파일 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
data = pd.read_table('ratings.txt')
print(f"전체 샘플의 수: {len(data)}")

data.head()


In [ ]:
# 데이터의 일부만 사용합시다
sample_data = data[:1000]

In [ ]:
# 데이터 전처리
# 한글, 영어, 숫자 및 ASCII 특수문자, 공백을 제외하고 삭제합니다.
def extract_ko_en_only(s: str):
    return re.sub(r"[^\x20-\x7Eㄱ-ㅎㅏ-ㅣ가-힣]+", "", s)

sample_data.loc[:, "document"] = sample_data["document"].apply(extract_ko_en_only)

# 불용어 정의

자주 등장하지만 분석에는 큰 도움이 되지 않는 형태소들을 토큰화 후 제거합니다.

In [ ]:
# 불용어 리스트 예시
stopwords = ["의", "가", "이", "은", "는", "을", "를", "이", "도", "으로", "로", "하", "다", "게", "음", "입니다", "습니다"]

In [ ]:
tokenizer = MeCab()
tokenized=[]
for sentence in sample_data['document']:
    temp = tokenizer.morphs(sentence) # 토큰화
    temp = [tok for tok in temp if not tok in stopwords] #불용어 제거
    tokenized.append(temp)

In [ ]:
# 데이터 예시
for tok_list in tokenized[:5]:
    print(tok_list)

In [ ]:
# Counter를 이용하여 토큰 등장 빈도수를 세봅시다.
counter = Counter()

for tok_list in tokenized:
    counter.update(tok_list)

In [ ]:
# Counter의 크기 = 등장한 단어의 수
print(len(counter))
# 빈도가 많은 10개의 단어 (와 빈도)를 출력해봅니다.
print(counter.most_common(10))

자연어처리에서는 **토큰** 하나에 대하여 **벡터** 하나를 할당합니다.

즉 우리가 <ins>고려하는 토큰 종류가 많아질수록</ins>, 머신이 <ins>기억해야 하는 벡터의 개수</ins>가 증가합니다.

따라서 머신의 성능을 고려하여 단어 집합 (Vocabulary)의 크기를 제한하고, 사전에 제외된 단어 및 형태소를 위한 토큰 (`<unk>`)을 정의합니다.

또 이러한 방식의 장점은, 실전에서 학습 데이터에서 등장하지 않았던 단어가 나타나더라도 오류없이 대응할 수 있다는 것입니다.

최종적으로 단어집합은 **단어에서 인덱스(번호)**, **인덱스에서 단어간의 매핑**으로 구성되며, 인덱스 번호는 추후 벡터집합 (행렬)의 위치 (행)을 가리키는데 사용됩니다. 

In [ ]:
# 파이썬 사전으로 vocabulary를 정의합니다.
word_to_index = {}
word_to_index["<unk>"] = 0 # 제외된 단어를 위한 토큰을 vocabulary에 추가합니다.
word_to_index["<pad>"] = 1 # 추후에 토큰의 빈칸을 나타낼 때 사용하는 토큰입니다.

#가장 자주 등장한 498개의 단어만 추가하여 크기가 총 500인 사전을 만듭시다.
mc_word = [c for c, _ in counter.most_common(498)]

#enumerate는 순서가 있는 자료형을 입력으로 넣으면 (인덱스, 원소)를 반환합니다.
word_to_index.update({key: idx + 2 for idx, key in enumerate(mc_word)})

# 확인
for key in list(word_to_index.keys())[:10]:
    print(f"단어:\t{key}\t인덱스:{word_to_index[key]}")

# 인덱스에서 단어로 복원하는 사전을 만듭니다.
index_to_word = list(word_to_index.keys())

In [ ]:
# 토큰 리스트에서 정수(index) 리스트로 변환합니다.
encoded = []
for line in tokenized:
    index_list = []
    for tok in line:
        if tok in word_to_index:
            index_list.append(word_to_index[tok])
        else:
            index_list.append(word_to_index["<unk>"])
    encoded.append(index_list)

# 확인해봅시다.
print(tokenized[0])
print(encoded[0])

# 패딩 (padding)

우리가 사용하는 문장의 길이가 제각각이며, 따라서 문장당 토큰의 개수도 달라질 수 있습니다.

이는 딥러닝 모델이 여러 문장을 동시에 처리 및 학습할 때 병렬화하는 걸림돌이 됩니다.

때문에 문장의 길이를 미리 맞춰주는 처리를 하는데, 의미 없는 토큰을 문장 뒤에 추가하는 과정을 패딩이라고 합니다.

이 예제에서는 모든 문장을 가장 긴 문장의 길이로 전부 맞춰주는 예제입니다.

In [ ]:
# 최대 토큰 길이
max_len = max([len(l) for l in encoded])

print(f"최대 길이: {max_len}")

In [ ]:
# 패딩
for line in encoded:
    line += [word_to_index["<pad>"]]*(max_len-len(line))

# 확인
print(encoded[0])